In [12]:
## Inserindo dados do atlas no data lake
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd



In [ ]:
# Todas as tabela do catálogo do atlas estão em csv, exceto: 

##SIAPE_ativos_economia_rural
##SIAPE_civis_ativos_ministerio

# Nessa pasta também será inserida no datalake a base da FLACSO

##FLACSO_acoes_afirmativas.csv 

In [2]:
client = bigquery.Client()

In [3]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')

In [ ]:
## Lista por tipo de arquivo

len(glob.glob('*.csv'))

In [4]:
df = pd.read_csv('FLACSO_acoes_afirmativas.csv', encoding='utf8', decimal=",")


In [5]:
df.drop(['id','marker_id'], axis=1, inplace=True)

In [7]:
len(df)

135

In [6]:
df.columns

Index(['id', 'marker_id', 'abrangencia', 'regiao', 'estado', 'cidade', 'Ano',
       'Legislação', 'regulamentacao', 'tipo_cota', 'comissionado',
       'percentual', 'pub_alvo', 'identificacao', 'comiss_ver', 'vigencia',
       'lat', 'lng'],
      dtype='object')

In [6]:
df.rename(columns={'descricao':'legislacao', 'regiao':'nome_regiao', 'estado':'sigla_uf', 
'identificacao': 'forma_identificacao','pub_alvo':'nomenclatura_legislacao', 'Ano':'ano','Legislação':'legislacao', 'comissionado':'flag_comissionado',
'comiss_ver':'flag_comiss_verificacao','cidade':'nome_municipio' },inplace=True)


In [7]:
## Ordenando algumas colunas: 

temp_cols=df.columns.tolist()
new_cols=temp_cols[4:5]+temp_cols[0:4] + temp_cols[5:]
df=df[new_cols]
df


,ano,abrangencia,nome_regiao,sigla_uf,nome_municipio,legislacao,regulamentacao,tipo_cota,flag_comissionado,percentual,nomenclatura_legislacao,forma_identificacao,flag_comiss_verificacao,vigencia,lat,lng
0,2002,municipal,sul,RS,Bagé,"Lei n.º 3.938, de 23.09.2002, reserva de 20% (...",NaN,Concurso público,0,20%,Afrodescendentes,não consta,0,vigente,-313.301,-541.005
1,2003,municipal,sul,RS,Pelotas,"Lei n.º 4.989, de 21.11.2003, dispõe sobre a r...","Decreto n° 6.211, de 27.09.19, institui o Regi...",Concurso público,0,20%,Afrodescendentes,Autodeclaração,1,vigente,-317.654,-523.376
2,2004,municipal,sul,RS,Viamão,"Lei n.º 3.210, de 23.01.2004, dispõe sobre a r...",NaN,Concurso público,0,44%,Afro-brasileiros,Autodeclaração,0,revogada,-300.722,-51.097
3,2014,municipal,nordeste,BA,Teixeira de Freitas,"Lei n.º 815, de 18.11.2014, dispõe sobre o est...",NaN,Concurso público e estágio profissional,0,25%,"Negros, negras e afrodescendentes",Autodeclaração,0,vigente,-17.54,-397.422
4,2014,municipal,nordeste,PB,Sapé,"Lei n.º 1.178, de 15.10.2014, dispõe sobre o e...",NaN,Concurso público,0,20%,"Negros, negras e afrodescendentes",autodeclaração,0,vigente,-709.616,-352.302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2018,municipal,nordeste,CE,Jaguaribe,"Lei n°1.422, de 26 de outubro de 2018, dispõe ...",NaN,Concurso público,0,NaN,Negros e afrodescendentes,autodeclaração,0,vigente,-589.273,-38.622
131,2021,estadual,norte,RO,Porto Velho,"Resolução n° 102 - CS/DPERO, de 02 de julho de...",NaN,Concurso público,0,20%,Negros,Autodeclaração,0,vigente,-876.116,-639.004
132,2021,estadual,sudeste,RJ,Rio de Janeiro,"Resolução DPGERJ, n° 1107, de 29 de setembro d...",NaN,Concurso Público e estagiário,0,20 a 30,Afrodescendentes,Não consta,0,vigente,-229.068,-431.729
133,2021,estadual,nordeste,PI,Teresina,"Lei n.º 7.626, de 11 de novembro de 2021, rese...",NaN,Concurso público e contratação temporária,0,25%,Negros e pardos,auto-declaração,0,vigente,-504.469,-427.662


In [11]:
len(df)

135

In [8]:
# Padronizando região

df['nome_regiao'] = df['nome_regiao'].replace({'sul':'Sul', 'norte': 'Norte','nordeste':'Nordeste','centro-oeste':'Centro-oeste','sudeste':'Sudeste'})

In [9]:
# Padronizando tipo_cota
df['tipo_cota'] = df['tipo_cota'].replace({'Concurso público':'Concurso público', 'concurso público': 'Concurso público','Concurso Público e estagiário':'Concurso público e estágio profissional',
'Concurso público e contratação temporária.':'Concurso público e contratação temporária','Sistema de pontuação diferenciado em concurso público':'Concurso público'})

In [14]:
len(df)

135

In [ ]:
 ## Padronizando com map
#mapping = {'Concurso público':'Concurso público', 'concurso público': 'Concurso público','Concurso Público e estagiário':'Concurso público e estágio profissional'}
#df['tipo_cota'] = df['tipo_cota'].map(mapping)

In [10]:
df['percentual'] = df['percentual'].replace({'não consta':'Não consta','20 a 30':'20% a 30%'})

In [17]:
df['percentual'].unique()

array(['20%', '44%', '25%', '10%', '5%', 'Não consta', '12%', '30%',
       'metas percentuais', '15%', '17%', nan, '20% a 30%'], dtype=object)

In [ ]:
#df_tipos= pd.DataFrame({'coluna':df.columns,'tipos':tipos})
#df_tipos


In [ ]:
#df_tipos.iloc[13,1]

In [ ]:
#schema = []

#for i in list(range(len(df_tipos))):
 
 #   x=  "bigquery.SchemaField('{}',".format(df_tipos.iloc[i,0])
    
    
  #  y = x+"'{}')".format(df_tipos.iloc[i,1])


#    schema.append(y)

In [27]:
df['nome_municipio']

0                     Bagé
1                  Pelotas
2                   Viamão
3      Teixeira de Freitas
4                     Sapé
              ...         
130              Jaguaribe
131            Porto Velho
132         Rio de Janeiro
133               Teresina
134                  Natal
Name: nome_municipio, Length: 135, dtype: object

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ano                      135 non-null    int64 
 1   abrangencia              135 non-null    object
 2   nome_regiao              135 non-null    object
 3   sigla_uf                 135 non-null    object
 4   nome_municipio           135 non-null    object
 5   legislacao               135 non-null    object
 6   regulamentacao           23 non-null     object
 7   tipo_cota                135 non-null    object
 8   flag_comissionado        135 non-null    int64 
 9   percentual               134 non-null    object
 10  nomenclatura_legislacao  135 non-null    object
 11  forma_identificacao      135 non-null    object
 12  flag_comiss_verificacao  135 non-null    int64 
 13  vigencia                 135 non-null    object
 14  lat                      135 non-null    o

In [13]:
## Trazendo nomes de municipios
municipio=bd.read_sql("select sigla_uf, id_municipio, nome from `basedosdados.br_bd_diretorios_brasil.municipio`", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)
df = df.merge(municipio, left_on='id_municipio', right_on='id_municipio')

Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 11099.38rows/s]


KeyError: 'id_municipio'

In [19]:
client = bigquery.Client()

In [20]:
dataset_ref = client.dataset('acoes_afirmativas')

In [ ]:
df.columns

In [21]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de implementação da legislação.'),
 bigquery.SchemaField('abrangencia','STRING',description='Se a abrangência é municipal, estadual, federal, nacional ou distrital.'),
 bigquery.SchemaField('nome_regiao','STRING',description='Nome da Região.'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Município.'),
 bigquery.SchemaField('legislacao','STRING',description='Número da legislação e detalhes.'),
 bigquery.SchemaField('regulamentacao','STRING',description='Detalhes sobre a regulamentação'),
 bigquery.SchemaField('tipo_cota','STRING',description='A cota vale para quais formas de ingresso.'),
 bigquery.SchemaField('flag_comissionado','INTEGER',description='A ação abrange cargos comissionados?'),
 bigquery.SchemaField('percentual','STRING',description='Percentual de vagas reservadas'),
 bigquery.SchemaField('nomenclatura_legislacao','STRING',description='Se a abrangência é municipal, estadual ou federal'),
 bigquery.SchemaField('forma_identificacao','STRING',description='Forma de identificação do público alvo na legislação. '),
 bigquery.SchemaField('flag_comiss_verificacao','INTEGER',description='Se há comissão de verificação.'),
 bigquery.SchemaField('vigencia','STRING',description='Se está vigente ou não.'),
 bigquery.SchemaField('lat','STRING',description='Latitude'),
 bigquery.SchemaField('lng','STRING',description='Longitude')]

In [22]:
table_ref = dataset_ref.table('FLACSO_acoes_afirmativas_grafico')

In [23]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [26]:
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=86606ccd-7bb3-4252-81d9-abf4b6240702>

In [25]:
#para excluir tabelas
client.delete_table('acoes_afirmativas.FLACSO_acoes_afirmativas_grafico', not_found_ok=True)

In [ ]:
df.dtypes


In [ ]:
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

# Método 2: Usando JSON:
https://cloud.google.com/bigquery/docs/schemas?hl=pt-br
